# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../Output/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chonchi,-42.6128,-73.8092,14.51,73,100,1.36,CL,1701020065
1,1,estacion coahuila,32.1939,-115.0017,16.04,22,85,5.82,MX,1701020066
2,2,anadyr,64.7500,177.4833,1.28,100,100,13.00,RU,1701020067
3,3,invercargill,-46.4000,168.3500,10.57,92,100,0.45,NZ,1701020067
4,4,polyarnyy,69.1989,33.4478,-11.59,90,93,2.51,RU,1701020068


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM",
                                               frame_width=700, frame_height=500,
                                               size="Humidity", scale=0.5, color="City")

# Display the map
humidity_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
# Ideal Weather Conditions:
    # Max temperature lower than 20 degrees but higher than 5
    # Wind speed less than 4.5 m/s
    # Cloudiness equal to 0

ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 20) &
                                    (city_data_df["Max Temp"] > 10) &
                                    (city_data_df["Wind Speed"] < 4.5) &
                                    (city_data_df["Cloudiness"] == 0)
                                   ]
    
# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,75,jinjiang,24.8198,118.5741,18.34,82,0,2.11,CN,1701020120
76,76,lompoc,34.6391,-120.4579,18.21,63,0,4.12,US,1701019896
111,111,yiyang,28.5892,112.3283,10.99,73,0,1.51,CN,1701020152
330,330,isla vista,34.4133,-119.8610,19.32,38,0,0.45,US,1701020331
375,375,murzuq,25.9155,13.9184,16.73,39,0,3.36,LY,1701020369
391,391,suez,29.9737,32.5263,18.68,31,0,3.40,EG,1701020382
417,417,magalia,39.8121,-121.5783,11.18,30,0,1.79,US,1701020321
467,467,saveh,35.0213,50.3566,14.85,36,0,3.72,IR,1701020436
471,471,fig tree,-33.0317,148.7511,12.97,70,0,1.24,AU,1701020439
475,475,ouarzazat,30.9189,-6.8934,19.16,6,0,2.06,MA,1701020441


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
75,jinjiang,CN,24.8198,118.5741,82,
76,lompoc,US,34.6391,-120.4579,63,
111,yiyang,CN,28.5892,112.3283,73,
330,isla vista,US,34.4133,-119.8610,38,
375,murzuq,LY,25.9155,13.9184,39,
391,suez,EG,29.9737,32.5263,31,
417,magalia,US,39.8121,-121.5783,30,
467,saveh,IR,35.0213,50.3566,36,
471,fig tree,AU,-33.0317,148.7511,70,
475,ouarzazat,MA,30.9189,-6.8934,6,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
jinjiang - nearest hotel: Metropolo Hotels
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
yiyang - nearest hotel: No hotel found
isla vista - nearest hotel: The Club & Guest House
murzuq - nearest hotel: فندق باريس - مرزق
suez - nearest hotel: فندق لوسيندا
magalia - nearest hotel: Ponderosa Gardens Motel
saveh - nearest hotel: هتل صدرا
fig tree - nearest hotel: No hotel found
ouarzazat - nearest hotel: Fint
delijan - nearest hotel: هتل سبز کارون


,City,Country,Lat,Lng,Humidity,Hotel Name
75,jinjiang,CN,24.8198,118.5741,82,Metropolo Hotels
76,lompoc,US,34.6391,-120.4579,63,Embassy Suites by Hilton Lompoc Central Coast
111,yiyang,CN,28.5892,112.3283,73,No hotel found
330,isla vista,US,34.4133,-119.8610,38,The Club & Guest House
375,murzuq,LY,25.9155,13.9184,39,فندق باريس - مرزق
391,suez,EG,29.9737,32.5263,31,فندق لوسيندا
417,magalia,US,39.8121,-121.5783,30,Ponderosa Gardens Motel
467,saveh,IR,35.0213,50.3566,36,هتل صدرا
471,fig tree,AU,-33.0317,148.7511,70,No hotel found
475,ouarzazat,MA,30.9189,-6.8934,6,Fint


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM",
                                        frame_width=700, frame_height=500,
                                        size="Humidity", scale=0.7, color="City",
                                        hover_cols=["City", "Country", "Hotel Name"])

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)